# DLA for Z2 lattice gauge theory Hamiltonian variational ansatz (sparse implementation)

In [ ]:
import numpy as np
import h5py
import jax
from fastdla.sparse_pauli_vector import SparsePauliVectorArray, SparsePauliVector
from fastdla.lie_closure import lie_closure, linear_independence
from fastdla.generators.z2lgt_hva import (z2lgt_hva_generators, z2lgt_gauss_projector,
                                          z2lgt_u1_projector, z2lgt_translation_projector,
                                          z2lgt_symmetry_eigenspace)

jax.config.update('jax_enable_x64', True)
SparsePauliVector.switch_impl('fast')

In [ ]:
data_file = ''
max_workers = 110

## 2 Fermions

In [3]:
num_fermions = 2
# Determine the charge sector (symmetry subspace) to investigate
gauss_eigvals = [1, -1, 1, -1]
u1_total_charge = 0
t_jphase = 0

In [4]:
# Full HVA generators
generators_full = z2lgt_hva_generators(num_fermions)
generators_full

SparsePauliVectorArray(5 vectors, num_qubits=8)

### Using SparsePauliVectors

In [5]:
# Compute the DLA of the full space
dla_full = lie_closure(generators_full, min_tasks=200, max_workers=max_workers, verbosity=2)
print(f'DLA dimension is {len(dla_full)}')

Starting with 10 commutators..
Evaluating 10/10 commutators for independence
Adding 35 commutators; total 35
Current DLA dimension: 10
Evaluating 35/35 commutators for independence
Adding 280 commutators; total 280
Current DLA dimension: 26
Evaluating 259/280 commutators for independence
Adding 7178 commutators; total 7199
Current DLA dimension: 123
Evaluating 3578/7199 commutators for independence
Adding 40702 commutators; total 44323
Current DLA dimension: 311
Evaluating 43609/44323 commutators for independence
Current DLA dimension: 311
Evaluating 714/714 commutators for independence
Current DLA dimension: 311
DLA dimension is 311


In [6]:
# Projectors
gauss_projector = z2lgt_gauss_projector(gauss_eigvals)
u1_projector = z2lgt_u1_projector(num_fermions, u1_total_charge)
t_projector = z2lgt_translation_projector(num_fermions, t_jphase)
symm_projector = gauss_projector @ u1_projector @ t_projector
# HVA generators and symmetry generators are simultaneously diagonalizable
# -> HVA generators can be projected onto the symmetry subspace by one-side application of the
# projectors. The resulting operators are the HVA generators in the subspace.
generators_projected = SparsePauliVectorArray([(op @ symm_projector).normalize() for op in generators_full])

In [7]:
# Refine the generators list in case some are linearly dependent in the subspace
generators_indep = SparsePauliVectorArray([generators_projected[0]])
for gen in generators_projected[1:]:
    if linear_independence(gen, generators_indep):
        generators_indep.append(gen)
print(f'{len(generators_indep)} generators are independent')

4 generators are independent


In [8]:
# Compute the DLA of the subspace
dla = lie_closure(generators_indep, min_tasks=200, max_workers=max_workers, verbosity=2)
print(f'Subspace DLA dimension is {len(dla)}')

Starting with 6 commutators..
Evaluating 6/6 commutators for independence
Adding 30 commutators; total 30
Current DLA dimension: 9
Evaluating 30/30 commutators for independence
Adding 264 commutators; total 264
Current DLA dimension: 25
Evaluating 262/264 commutators for independence
Adding 1470 commutators; total 1472
Current DLA dimension: 60
Evaluating 1234/1472 commutators for independence
Adding 183 commutators; total 421
Current DLA dimension: 63
Evaluating 238/421 commutators for independence
Current DLA dimension: 63
Evaluating 183/183 commutators for independence
Current DLA dimension: 63
Subspace DLA dimension is 63


### Using matrices

In [9]:
generators_csr = [gen.to_matrix(sparse=True) for gen in generators_full]
symm_eigenspace = z2lgt_symmetry_eigenspace(gauss_eigvals, u1_total_charge, t_jphase)
generators_reduced = np.array([symm_eigenspace.conjugate().T @ gen.dot(symm_eigenspace)
                               for gen in generators_csr])

In [10]:
# Refine the generators list in case some are linearly dependent in the subspace
indep_indices = [0]
for igen, gen in enumerate(generators_reduced[1:]):
    if linear_independence(gen, generators_reduced[indep_indices]):
        indep_indices.append(igen + 1)
generators_mat_indep = generators_reduced[indep_indices]
print(f'{len(indep_indices)} generators are independent')

4 generators are independent


In [11]:
# Compute the DLA of the subspace
dla_arr = lie_closure(generators_mat_indep, verbosity=2)
print(f'Subspace DLA dimension is {len(dla_arr)}')

Basis size 63; 2000th commutator [b[62], b[47]]
Found 59 new ops in 1.52s
Subspace DLA dimension is 63


In [12]:
# Compute the DLA dimension of the subspace
dla_dimonly = lie_closure(generators_mat_indep, keep_original=False, verbosity=2)
print(f'Subspace DLA dimension is {len(dla_dimonly)}')

Basis size 63; 2000th commutator [b[62], b[47]]
Found 59 new ops in 0.56s
Subspace DLA dimension is 63


In [13]:
if data_file:
    with h5py.File(data_file, 'a') as out:
        if 'nf=2' not in out:
            nf_group = out.create_group('nf=2')
            for gname, oplist in [
                ('generators_full', generators_full),
                ('dla_full', dla_full),
                ('generators_symm0', generators_indep),
                ('dla_symm0', dla)
            ]:
                group = nf_group.create_group(gname)
                group.create_dataset('indices', data=oplist.indices)
                group.create_dataset('coeffs', data=oplist.coeffs)
                group.create_dataset('indptr', data=oplist.ptrs)

            nf_group.create_dataset('dla_symm0/gauss_eigvals', data=gauss_eigvals)
            nf_group.create_dataset('dla_symm0/u1_total_charge', data=u1_total_charge)
            nf_group.create_dataset('dla_symm0/t_jphase', data=t_jphase)

## 3 Fermions

In [14]:
num_fermions = 3
# Determine the charge sector (symmetry subspace) to investigate
gauss_eigvals = [1, -1, 1, -1, 1, -1]
u1_total_charge = 0
t_jphase = 0

In [15]:
# Full HVA generators
generators_full = z2lgt_hva_generators(num_fermions)
generators_full

SparsePauliVectorArray(5 vectors, num_qubits=12)

### Using SparsePauliVectors

In [16]:
# Compute the DLA of the full space (takes too long)
# dla_full = lie_closure(generators_full, min_tasks=200, max_workers=max_workers, verbosity=2)
# print(f'DLA dimension is {len(dla_full)}')

In [17]:
# Projectors
gauss_projector = z2lgt_gauss_projector(gauss_eigvals)
u1_projector = z2lgt_u1_projector(num_fermions, u1_total_charge)
t_projector = z2lgt_translation_projector(num_fermions, t_jphase)
symm_projector = gauss_projector @ u1_projector @ t_projector
# HVA generators and symmetry generators are simultaneously diagonalizable
# -> HVA generators can be projected onto the symmetry subspace by one-side application of the
# projectors. The resulting operators are the HVA generators in the subspace.
generators_projected = SparsePauliVectorArray([(op @ symm_projector).normalize() for op in generators_full])

In [18]:
# Refine the generators list in case some are linearly dependent in the subspace
generators_indep = SparsePauliVectorArray([generators_projected[0]])
for gen in generators_projected[1:]:
    if linear_independence(gen, generators_indep):
        generators_indep.append(gen)
print(f'{len(generators_indep)} generators are independent')

4 generators are independent


In [19]:
# Compute the DLA of the subspace
dla = lie_closure(generators_indep, min_tasks=200, max_workers=max_workers, verbosity=2)
print(f'Subspace DLA dimension is {len(dla)}')

Starting with 6 commutators..
Evaluating 6/6 commutators for independence
Adding 30 commutators; total 30
Current DLA dimension: 9
Evaluating 30/30 commutators for independence
Adding 264 commutators; total 264
Current DLA dimension: 25
Evaluating 203/264 commutators for independence
Adding 6141 commutators; total 6202
Current DLA dimension: 114
Evaluating 201/6202 commutators for independence
Adding 11704 commutators; total 17705
Current DLA dimension: 191
Evaluating 201/17705 commutators for independence
Adding 9821 commutators; total 27325
Current DLA dimension: 237
Evaluating 204/27325 commutators for independence
Adding 2415 commutators; total 29536
Current DLA dimension: 247
Evaluating 201/29536 commutators for independence
Adding 994 commutators; total 30329
Current DLA dimension: 251
Evaluating 202/30329 commutators for independence
Adding 503 commutators; total 30630
Current DLA dimension: 253
Evaluating 201/30630 commutators for independence
Adding 253 commutators; total 3068

### Using matrices

In [20]:
generators_csr = [gen.to_matrix(sparse=True) for gen in generators_full]
symm_eigenspace = z2lgt_symmetry_eigenspace(gauss_eigvals, u1_total_charge, t_jphase)
generators_reduced = np.array([symm_eigenspace.conjugate().T @ gen.dot(symm_eigenspace)
                               for gen in generators_csr])

In [21]:
# Refine the generators list in case some are linearly dependent in the subspace
indep_indices = [0]
for igen, gen in enumerate(generators_reduced[1:]):
    if linear_independence(gen, generators_reduced[indep_indices]):
        indep_indices.append(igen + 1)
generators_mat_indep = generators_reduced[indep_indices]
print(f'{len(indep_indices)} generators are independent')

4 generators are independent


In [23]:
# Compute the DLA of the subspace
dla_arr = lie_closure(generators_mat_indep, verbosity=2)
print(f'Subspace DLA dimension is {len(dla_arr)}')

Basis size 239; 2000th commutator [b[62], b[47]]
Basis size 251; 4000th commutator [b[88], b[84]]
Basis size 255; 6000th commutator [b[109], b[5]]
Basis size 255; 10000th commutator [b[140], b[130]]
Basis size 255; 12000th commutator [b[154], b[65]]
Basis size 255; 14000th commutator [b[166], b[139]]
Basis size 255; 16000th commutator [b[178], b[69]]
Basis size 255; 18000th commutator [b[189], b[45]]
Basis size 255; 20000th commutator [b[199], b[100]]
Basis size 255; 22000th commutator [b[209], b[55]]
Basis size 255; 24000th commutator [b[218], b[129]]
Basis size 255; 26000th commutator [b[227], b[122]]
Basis size 255; 28000th commutator [b[236], b[34]]
Basis size 255; 30000th commutator [b[244], b[110]]
Basis size 255; 32000th commutator [b[252], b[122]]
Found 251 new ops in 13.04s
Subspace DLA dimension is 255


In [24]:
# Compute the DLA dimension of the subspace
dla_dimonly = lie_closure(generators_mat_indep, keep_original=False, verbosity=2)
print(f'Subspace DLA dimension is {len(dla_dimonly)}')

Basis size 239; 2000th commutator [b[62], b[47]]
Basis size 251; 4000th commutator [b[88], b[84]]
Basis size 255; 6000th commutator [b[109], b[5]]
Basis size 255; 10000th commutator [b[140], b[130]]
Basis size 255; 12000th commutator [b[154], b[65]]
Basis size 255; 14000th commutator [b[166], b[139]]
Basis size 255; 16000th commutator [b[178], b[69]]
Basis size 255; 18000th commutator [b[189], b[45]]
Basis size 255; 20000th commutator [b[199], b[100]]
Basis size 255; 22000th commutator [b[209], b[55]]
Basis size 255; 24000th commutator [b[218], b[129]]
Basis size 255; 26000th commutator [b[227], b[122]]
Basis size 255; 28000th commutator [b[236], b[34]]
Basis size 255; 30000th commutator [b[244], b[110]]
Basis size 255; 32000th commutator [b[252], b[122]]
Found 251 new ops in 6.04s
Subspace DLA dimension is 255


In [25]:
if data_file:
    with h5py.File(data_file, 'a') as out:
        if 'nf=2' not in out:
            nf_group = out.create_group('nf=3')
            for gname, oplist in [
                ('generators_full', generators_full),
                ('dla_full', dla_full),
                ('generators_symm0', generators_indep),
                ('dla_symm0', dla)
            ]:
                group = nf_group.create_group(gname)
                group.create_dataset('indices', data=oplist.indices)
                group.create_dataset('coeffs', data=oplist.coeffs)
                group.create_dataset('indptr', data=oplist.ptrs)

            nf_group.create_dataset('dla_symm0/gauss_eigvals', data=gauss_eigvals)
            nf_group.create_dataset('dla_symm0/u1_total_charge', data=u1_total_charge)
            nf_group.create_dataset('dla_symm0/t_jphase', data=t_jphase)